# Making the initial timetable

In [2]:
import pandas as pd
import numpy as np

In [3]:
proc_tt = pd.read_excel("Processed Data/encoded_timetable_data.xlsx").copy()
print(proc_tt.columns)
print("Shape of dataframe: ", proc_tt.shape)

Index(['Course', 'Code', 'Linked Course', 'Activity', 'Activity Type Name',
       'Planned Size', 'Real Size', 'Delivery Semester',
       'Teaching Week Pattern', 'Number Of Teaching Weeks', 'Scheduled Days',
       'Scheduled Start Time', 'Scheduled End Time', 'Duration', 'Total Hours',
       'Zone Name', 'room_name', 'Delivery Period', 'Normal Year Taken',
       'building', 'Abbreviated Course Name', 'time_periods', 'duration_hours',
       'total_duration_hours', 'course_classtype', 'ws_groups',
       'no_of_agg_ws_groups', 'aggregated_ws_groups', 'sem1_pattern_one_hot',
       'sem2_pattern_one_hot'],
      dtype='object')
Shape of dataframe:  (1259, 30)


## Cleaning data

In [4]:
# Drop MIGS course as we are not considering it for SoM
proc_tt.drop(proc_tt.index[proc_tt.Code == "MATH11220"].tolist(), inplace = True)

## Choosing weeks in semester 1 and semester 2 for timetabling

In [5]:
sem1_tt = proc_tt[proc_tt["sem1_pattern_one_hot"].str.contains('1')]
sem2_tt = proc_tt[proc_tt["sem2_pattern_one_hot"].str.contains('1')]
s2w3_tt = sem2_tt[sem2_tt["Teaching Week Pattern"].str.contains('s2w3')]
s2w4_tt = sem2_tt[sem2_tt["Teaching Week Pattern"].str.contains('s2w4')]

In [6]:
init_tt = np.zeros((9, 5), dtype = "O")

days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
periods = ['1', '2', '3', '4', '5', '6', '7', '8', '9']

In [7]:
# s2w3_Mon_1 = s2w3_tt[(s2w3_tt["Scheduled Days"] == "Monday") & (s2w3_tt["time_periods"].str.contains('1'))]
# s2w3_Tue_8 = s2w3_tt[(s2w3_tt["Scheduled Days"] == "Tuesday") & (s2w3_tt["time_periods"].str.contains('8'))]
# for row in s2w3_Mon_1.itertuples():
# #     print(row)
# to_timetable = s2w3_Mon_1.drop_duplicates(subset = ["course_classtype", "room_name"])
# classes_rooms = []
# for row in to_timetable.itertuples():
#     print([row.course_classtype, row.room_name])
#     classes_rooms.append([row.course_classtype, row.room_name])
# init_tt[0,0] = classes_rooms
count = 0
count_check = 0
for d, day in enumerate(days):
    for t, period in enumerate(periods):
        print(day, period)
        s2w3_day_time_tt = s2w3_tt[(s2w3_tt["Scheduled Days"] == day) & (s2w3_tt["time_periods"].str.contains(period))]
        s2w3_dt_tt_unique = s2w3_day_time_tt.drop_duplicates(subset = ["course_classtype", "room_name"])
        count_check += s2w3_dt_tt_unique.shape[0]
        classes_rooms = []
        for row in s2w3_dt_tt_unique.itertuples():
            classes_rooms.append([row.course_classtype, row.room_name])
            count += 1
        
        init_tt[t, d] = classes_rooms





Monday 1
Monday 2
Monday 3
Monday 4
Monday 5
Monday 6
Monday 7
Monday 

8
Monday 9
Tuesday 1
Tuesday 2
Tuesday 3
Tuesday 4
Tuesday 5
Tuesday 6
Tuesday 7
Tuesday 8
Tuesday 9
Wednesday 1
Wednesday 2
Wednesday 3
Wednesday 4
Wednesday 5
Wednesday 6
Wednesday 7
Wednesday 8
Wednesday 9
Thursday 1
Thursday 2
Thursday 3
Thursday 4
Thursday 5
Thursday 6
Thursday 7
Thursday 8
Thursday 9
Friday 1
Friday 2
Friday 3
Friday 4
Friday 5
Friday 6
Friday 7
Friday 8
Friday 9


In [8]:
init_tt

array([[list([['MATH10077_lec', 'JCMB_5327'], ['MATH10069_lec', 'NUC_1.14 - Oak Lecture Theatre'], ['MATH11181_lec', 'JCMB_Lecture Theatre C'], ['MATH08059_ws', 'AT_M2 - Teaching Studio'], ['MATH11197_lec', 'JCMB_5326']]),
        list([['MATH08058_ws', 'AT_M2 - Teaching Studio'], ['MATH08058_ws', '40GS_LG.07 - Teaching Studio'], ['MATH11240_lec', 'JCMB_5327'], ['MATH11028_lec', 'JCMB_Lecture Theatre C'], ['MATH11238_lec', 'JCMB_1501']]),
        list([['MATH08075_ws', 'NUC_2.03 - Hawthorn Teaching Studio'], ['MATH11192_lec', 'JCMB_Lecture Theatre B'], ['MATH11205_ws', 'MH_G.07 - Teaching Studio'], ['MATH11205_ws', 'JCMB_1501'], ['MATH10093_cws', 'MH_LG.12 - Open Access Computer Lab'], ['MATH10093_cws', 'MH_G.09 - Teaching Studio'], ['MATH10093_cws', 'JCMB_5205']]),
        list([['MATH10077_lec', 'JCMB_6206'], ['MATH10069_lec', 'NUC_1.14 - Oak Lecture Theatre'], ['MATH08059_lec', '40GS_Lecture Theatre C'], ['MATH11188_ws', 'JCMB_3217 - Teaching Studio']]),
        list([['MATH08065_cw

In [19]:
print(count)
s2w3_copy = s2w3_tt.copy()
s2w3_copy["time_periods"] = s2w3_tt["time_periods"].str.split(',')
s2w3_copy = s2w3_copy.explode("time_periods")
s2w3_copy.reset_index(drop=True, inplace=True)
s2w3_copy.drop_duplicates(subset = ["course_classtype", "room_name", "Scheduled Days", "time_periods"])

229


,Course,Code,Linked Course,Activity,Activity Type Name,Planned Size,Real Size,Delivery Semester,Teaching Week Pattern,Number Of Teaching Weeks,...,Abbreviated Course Name,time_periods,duration_hours,total_duration_hours,course_classtype,ws_groups,no_of_agg_ws_groups,aggregated_ws_groups,sem1_pattern_one_hot,sem2_pattern_one_hot
0,Advanced Methods of Applied Mathematics,MATH10086,NaN,MATH4 Adv Methods Applied Maths - Lecture,Lecture,65,57,SEM 2,"s2w1, s2w2, s2w3, s2w4, s2w5, s2w6, s2w7, s2w8...",11,...,AMAM,2,1.0,11.0,MATH10086_lec,NaN,1,NaN,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"
1,Advanced Methods of Applied Mathematics,MATH10086,NaN,MATH4 Adv Methods Applied Maths - Lecture,Lecture,65,57,SEM 2,"s2w1, s2w2, s2w3, s2w4, s2w5, s2w6, s2w7, s2w8...",11,...,AMAM,2,1.0,11.0,MATH10086_lec,NaN,1,NaN,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"
2,Advanced Methods of Applied Mathematics,MATH10086,NaN,MATH4 Adv Methods Applied Maths - Lecture,Lecture,65,57,SEM 2,"s2w1, s2w2, s2w3, s2w4, s2w5, s2w6, s2w7, s2w8...",11,...,AMAM,3,1.0,11.0,MATH10086_lec,NaN,1,NaN,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"
3,Algebraic Geometry,MATH11120,NaN,MATH5 Algebraic Geometry - Lecture,Lecture,35,42,SEM 2,"s2w1, s2w2, s2w3, s2w4, s2w5, s2w6, s2w7, s2w8...",11,...,AG,2,1.0,11.0,MATH11120_lec,NaN,1,NaN,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"
4,Algebraic Geometry,MATH11120,NaN,MATH5 Algebraic Geometry - Lecture,Lecture,35,42,SEM 2,"s2w1, s2w2, s2w3, s2w4, s2w5, s2w6, s2w7, s2w8...",11,...,AG,2,1.0,11.0,MATH11120_lec,NaN,1,NaN,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646,Topics in Applied Operational Research,MATH11183,NaN,MATHO Topics in Applied Operational Research -...,Workshop,70,53,SEM 2 - Odd weeks only,"s2w1, s2w3, s2w5, s2w7, s2w9, s2w11",6,...,TAOR,7,2.0,12.0,MATH11183_ws,MATH11183_ws_00,1,MATH11183_ws_00,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0","1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1"
647,Topics in Mathematical Biology,MATH10083,NaN,MATH4 Topics in Maths Biology - Lecture,Lecture,5,3,SEM 2,"s2w1, s2w2, s2w3, s2w4, s2w5, s2w6, s2w7, s2w8...",11,...,TMB,6,2.0,22.0,MATH10083_lec,NaN,1,NaN,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"
648,Topics in Mathematical Biology,MATH10083,NaN,MATH4 Topics in Maths Biology - Lecture,Lecture,5,3,SEM 2,"s2w1, s2w2, s2w3, s2w4, s2w5, s2w6, s2w7, s2w8...",11,...,TMB,7,2.0,22.0,MATH10083_lec,NaN,1,NaN,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"
649,Topics in Mathematical Physics A,MATH11227,NaN,MATH5 Topics in Maths Physics A - Lecture,Lecture,40,21,SEM 2,"s2w1, s2w2, s2w3, s2w4, s2w5, s2w6, s2w7, s2w8...",11,...,TMPA,6,2.0,22.0,MATH11227_lec,NaN,1,NaN,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"
